In [4]:
import pandas as pd
df=pd.read_csv('mon_data.csv')
df.dropna(subset=['text_image'],inplace=True)

filename          0
url             796
description     796
type              0
filepath          0
scraped           0
old_filename    796
old_filepath      0
text_image        0
dtype: int64

In [5]:
df=df.iloc[:,[-1,3]]
df

,text_image,type
0,FACTURE\n\nLOGO\n\nJoanna Binet\n48 Coubertin\...,0
1,Joanna Binet\n48 Coubertin\n31400 Paris\n\nFAC...,0
2,FACTURE\n\nMon entreprise : Nom de la société\...,0
3,Joanna Binet\n48 Coubertin\n31400 Paris\n\nFAC...,0
4,Payer en ligne >\nFACTURE No\n\nSFIDELI\n\nF/0...,0
...,...,...
1603,Minois Retail Merchants Association\n\n36 Sout...,16
1604,FEMALE\n\n3\n2 :\n1\n\nMALE\n\n3-\n2\n\n3\nS\n...,20
1605,(B&W) PROTECTED BY MINNESOTA TOBACCO LITIGATIO...,14
1606,FF\n\nPrincipal Investigator/Program Director ...,11


In [6]:
import re
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = stopwords.words('english')
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    w = unicode_to_ascii(str(w).lower().strip())
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!]+", " ", w)
    w = re.sub(r'\b\w{0,2}\b', '', w)

    # remove stopword
    mots = word_tokenize(w.strip())
    mots = [mot for mot in mots if mot not in stop_words]
    return ' '.join(mots).strip()



In [7]:
df.text_image = df.text_image.apply(lambda x :preprocess_sentence(x))


In [82]:
from gensim.models import Word2Vec
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     vector_size=200,
                     alpha=0.03, 
                     negative=10,
                     seed=1)


In [83]:
from nltk.corpus import stopwords
from nltk.tokenize.regexp import  RegexpTokenizer
stop_words = set(stopwords.words('french'))
stop_words.update(["." , ",",':'])
tokenizer=RegexpTokenizer(("[a-zA-Zé]{3,}"))
def stop_words_filtering(liste):
    liste_new=[]
    for i in liste:
        if i not in stop_words:
            liste_new.append(i)
        else:
            continue 
    return liste_new
sentences=[]
for txt in df.text_image:
    mots=stop_words_filtering(tokenizer.tokenize(str(txt).lower()))
    sentences.append(mots)

In [84]:
w2v_model.build_vocab(sentences, progress_per=100000)
words = list(w2v_model.wv.index_to_key)
print('Vocabulary size: %d' % len(words))
print(words)

Vocabulary size: 1106
['date', 'total', 'tobacco', 'new', 'research', 'net', 'source', 'nom', 'montant', 'non', 'university', 'salaire', 'cotisations', 'base', 'one', 'may', 'smoking', 'taux', 'valeur', 'cigarette', 'revenu', 'sociale', 'year', 'code', 'paie', 'name', 'health', 'facture', 'www', 'securite', 'cigarettes', 'heures', 'number', 'bulletin', 'csg', 'employeur', 'service', 'impot', 'paris', 'york', 'information', 'smoke', 'state', 'brut', 'company', 'payer', 'carte', 'imposable', 'use', 'american', 'page', 'per', 'study', 'rue', 'two', 'inc', 'time', 'program', 'brand', 'industrydocuments', 'https', 'chomage', 'adresse', 'paiement', 'also', 'tva', 'report', 'complementaire', 'data', 'contributions', 'years', 'national', 'part', 'assurance', 'medical', 'medicine', 'product', 'public', 'sante', 'would', 'francaise', 'telephone', 'retraite', 'general', 'smokers', 'department', 'please', 'high', 'amount', 'cancer', 'subject', 'form', 'office', 'description', 'address', 'client', 

In [102]:
v=[]
df_n=pd.DataFrame(columns=['text'])

In [104]:

for e,j in enumerate(df.text_image):
    mots=stop_words_filtering(tokenizer.tokenize(str(j).lower()))
    for i in mots:
        if i not in words:
             
            continue  
       
        else:
            v.append(i)
    df_n.loc[e]=' '.join(v)
            
    v=[]
    
            
#mot_vu=w2v_model.wv['facture']


In [105]:
df_n.text[3]
df.text_image.shape


(1566,)

In [131]:
import numpy as np
total_donnes=[]
text_total=np.zeros([200])
for txt in df_n.text:
    for mots in txt.split():
        text_total=text_total+w2v_model.wv[mots]
    total_donnes.append(text_total/len(txt.split()))
    text_total=np.zeros([200])
    
        

C:\Users\sadoun\AppData\Local\Temp\ipykernel_10504\1465323158.py:7: RuntimeWarning: invalid value encountered in true_divide
  total_donnes.append(text_total/len(txt.split()))


In [120]:
total_donnes=pd.DataFrame(np.array(total_donnes))
total_donnes['type']=df.type

In [121]:
total_donnes.shape
total_donnes.dropna(inplace=True)
total_donnes.head()

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,type
0,-1.756984,0.317420,-0.248904,0.226631,-0.234701,0.390436,-0.086576,1.326710,-0.363297,0.073002,...,-0.311198,0.572089,-0.305657,1.214558,-0.891684,0.855939,-0.860518,-0.631233,-0.090971,0.0
1,-1.895900,0.026922,-0.241642,0.276303,-0.322610,0.510605,-0.262356,1.489211,-0.371322,0.160381,...,-0.182875,0.429457,-0.602394,1.549363,-0.563384,0.728003,-0.681514,-0.806399,-0.143780,0.0
2,0.157067,1.079796,-0.697124,-0.058355,-0.645184,-0.045530,1.048800,1.414565,0.277924,0.420650,...,-0.539992,-0.267890,-1.782511,-0.383777,0.113479,-0.458142,-0.062219,0.934816,-0.763592,0.0
3,-1.895900,0.026922,-0.241642,0.276303,-0.322610,0.510605,-0.262356,1.489211,-0.371322,0.160381,...,-0.182875,0.429457,-0.602394,1.549363,-0.563384,0.728003,-0.681514,-0.806399,-0.143780,0.0
4,-0.119172,0.430006,-0.700115,0.816359,0.739402,1.115554,0.147204,1.739983,-0.074337,-0.277010,...,-0.208327,-0.709420,0.094753,0.992594,-0.952163,0.984138,-1.157638,0.039770,-0.628345,0.0


In [122]:
data=total_donnes.drop('type',axis=1)

target=total_donnes.type

In [123]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test=train_test_split(data,target,test_size=0.2,random_state=1234)

In [68]:
"""from sklearn import svm,model_selection,preprocessing
scaler=preprocessing.StandardScaler().fit(X_train)
X_train_scaled=scaler.transform(X_train)
X_test_scaled=scaler.transform(X_test)"""

In [133]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D, Dropout,Activation
model=Sequential()
model.add(Dense(units = 256,input_shape=(200,)))
model.add((Activation('relu')))
model.add(Dropout(0.3))
model.add(Dense(units = 23, activation = "softmax"))
model.summary()
#clf=linear_model.LogisticRegression(C=)
#clf.fit(X_train,y_train)
#ac=AdaBoostClassifier(base_estimator=dtc,n_estimators=400)
#ac.fit(X_train, y_train)


Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_26 (Dense)            (None, 256)               51456     
                                                                 
 dense_27 (Dense)            (None, 256)               65792     
                                                                 
 activation_12 (Activation)  (None, 256)               0         
                                                                 
 dropout_12 (Dropout)        (None, 256)               0         
                                                                 
 dense_28 (Dense)            (None, 23)                5911      
                                                                 
Total params: 123,159
Trainable params: 123,159
Non-trainable params: 0
_________________________________________________________________


In [136]:
model.compile(loss="sparse_categorical_crossentropy",optimizer = "adam", metrics = ["accuracy"])
model.fit(X_train,y_train,epochs = 50, batch_size = 32,validation_data = [X_test, y_test])

Epoch 1/100
33/33 [==============================] - 1s 10ms/step - loss: 0.1935 - accuracy: 0.9591 - val_loss: 4.7364 - val_accuracy: 0.4806
Epoch 2/100
33/33 [==============================] - 0s 3ms/step - loss: 0.1839 - accuracy: 0.9514 - val_loss: 4.6234 - val_accuracy: 0.5233
Epoch 3/100
33/33 [==============================] - 0s 4ms/step - loss: 0.1649 - accuracy: 0.9582 - val_loss: 4.6789 - val_accuracy: 0.5155
Epoch 4/100
33/33 [==============================] - 0s 3ms/step - loss: 0.1294 - accuracy: 0.9669 - val_loss: 4.8675 - val_accuracy: 0.5194
Epoch 5/100
33/33 [==============================] - 0s 3ms/step - loss: 0.1469 - accuracy: 0.9611 - val_loss: 4.8685 - val_accuracy: 0.5349
Epoch 6/100
33/33 [==============================] - 0s 3ms/step - loss: 0.1496 - accuracy: 0.9611 - val_loss: 4.7244 - val_accuracy: 0.5349
Epoch 7/100
33/33 [==============================] - 0s 3ms/step - loss: 0.1627 - accuracy: 0.9621 - val_loss: 4.7790 - val_accuracy: 0.5078
Epoch 8/100


Epoch 59/100
33/33 [==============================] - 0s 3ms/step - loss: 0.1180 - accuracy: 0.9737 - val_loss: 6.8090 - val_accuracy: 0.5155
Epoch 60/100
33/33 [==============================] - 0s 4ms/step - loss: 0.1209 - accuracy: 0.9747 - val_loss: 6.8636 - val_accuracy: 0.5116
Epoch 61/100
33/33 [==============================] - 0s 3ms/step - loss: 0.1134 - accuracy: 0.9660 - val_loss: 6.8130 - val_accuracy: 0.5194
Epoch 62/100
33/33 [==============================] - 0s 3ms/step - loss: 0.1089 - accuracy: 0.9640 - val_loss: 7.0156 - val_accuracy: 0.4961
Epoch 63/100
33/33 [==============================] - 0s 3ms/step - loss: 0.1190 - accuracy: 0.9669 - val_loss: 7.1962 - val_accuracy: 0.5116
Epoch 64/100
33/33 [==============================] - 0s 4ms/step - loss: 0.0949 - accuracy: 0.9747 - val_loss: 7.1488 - val_accuracy: 0.5155
Epoch 65/100
33/33 [==============================] - 0s 3ms/step - loss: 0.1050 - accuracy: 0.9747 - val_loss: 7.1304 - val_accuracy: 0.5194
Epoch 

In [137]:
model.evaluate(X_test, y_test)

9/9 [==============================] - 0s 2ms/step - loss: 8.1956 - accuracy: 0.4845


[8.195639610290527, 0.4844961166381836]